In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import torchvision
from torchvision import datasets, models, transforms

In [3]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from session import Session, TrainingSchedule
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import Validator
from callbacks import OneHotAccuracy
import Datasets.ImageClassifierData as ImageClassifierData

In [5]:
torch.cuda.is_available(), torch.cuda.get_device_name(0), torch.backends.cudnn.enabled

(True, 'GeForce GTX 770', True)

In [6]:
#DATA_PATH = Path('data/hymenoptera_data')
DATA_PATH = Path('data/dogscats')

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data = ImageClassifierData.from_paths(DATA_PATH, 16, data_transforms)

In [8]:
print(next(iter(data['train'])))

[
( 0 , 0 ,.,.) = 
 -1.7412 -1.7240 -1.7069  ...  -0.6965 -0.7308 -0.7308
 -1.7754 -1.7925 -1.7412  ...  -0.7137 -0.7137 -0.7137
 -1.7754 -1.7925 -1.7583  ...  -0.7137 -0.7137 -0.7137
           ...             ⋱             ...          
  0.3481  0.3309  0.3138  ...   0.8961  0.8618  0.8618
  0.2796  0.2796  0.3138  ...   0.8961  0.8618  0.8618
  0.2282  0.2453  0.2796  ...   0.8961  0.8618  0.8618

( 0 , 1 ,.,.) = 
 -1.8256 -1.8081 -1.7731  ...  -0.9853 -1.0203 -1.0203
 -1.8431 -1.8431 -1.7731  ...  -1.0028 -1.0028 -1.0028
 -1.8256 -1.8081 -1.7731  ...  -1.0028 -1.0028 -1.0028
           ...             ⋱             ...          
 -0.0399 -0.0574 -0.0749  ...   1.0105  1.0105  1.0105
 -0.1099 -0.1099 -0.0749  ...   1.0105  1.0105  1.0105
 -0.1625 -0.1450 -0.1099  ...   1.0105  1.0105  1.0105

( 0 , 2 ,.,.) = 
 -1.5779 -1.5604 -1.5256  ...  -1.0027 -1.0376 -1.0376
 -1.5953 -1.5953 -1.5430  ...  -1.0201 -1.0201 -1.0201
 -1.5779 -1.5779 -1.5430  ...  -1.0201 -1.0201 -1.0201
          

In [9]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    figure, ax = plt.subplots()
    ax.imshow(inp)


'''
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])
'''

"\n# Get a batch of training data\ninputs, classes = next(iter(dataloaders['train']))\n\n# Make a grid from batch\nout = torchvision.utils.make_grid(inputs)\n\nimshow(out, title=[class_names[x] for x in classes])\n"

In [10]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft.fc = nn.Sequential(
    nn.BatchNorm1d(num_features=num_ftrs),
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, 2),
    nn.LogSoftmax(dim=1)
)

In [11]:
model_ft

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNo

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=1e-3)

In [13]:
sess = Session(model_ft, criterion, optimizer)
sess.freeze()

In [14]:
lr_scheduler = CosAnneal(len(data['train']), T_mult=2)
validator = Validator(data['valid'], OneHotAccuracy())

In [15]:
schedule = TrainingSchedule(data['train'], callbacks=[lr_scheduler, validator])

In [ ]:
lr_find(sess, data['train'], start_lr=1e-5)

Variable containing:
-0.7498 -0.6395
-0.3568 -1.2038
-0.5389 -0.8755
-1.0612 -0.4247
-1.0009 -0.4582
-1.1045 -0.4025
-0.6915 -0.6948
-0.9536 -0.4867
-1.3723 -0.2924
-0.3144 -1.3101
-0.5276 -0.8917
-0.4604 -0.9971
-0.5142 -0.9113
-0.2769 -1.4195
-0.4512 -1.0130
-0.3667 -1.1810
[torch.cuda.FloatTensor of size 16x2 (GPU 0)]
 Variable containing:
 0
 1
 0
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 1
 1
 0
[torch.cuda.LongTensor of size 16 (GPU 0)]

Variable containing:
-0.3692 -1.1753
-1.1678 -0.3726
-0.9447 -0.4923
-0.9818 -0.4694
-0.3982 -1.1133
-0.1772 -1.8178
-0.5191 -0.9040
-1.0574 -0.4267
-0.3084 -1.3265
-0.4395 -1.0338
-0.4256 -1.0595
-1.5019 -0.2519
-0.5069 -0.9221
-0.6064 -0.7881
-0.5844 -0.8152
-0.6844 -0.7020
[torch.cuda.FloatTensor of size 16x2 (GPU 0)]
 Variable containing:
 1
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 0
 1
 1
[torch.cuda.LongTensor of size 16 (GPU 0)]

Variable containing:
-1.0585 -0.4262
-1.0593 -0.4257
-0.9228 -0.5065
-0.6526 -0.7355
-0.4981 -0.9356
-0.5721 -0.8309
-0.3960 -

In [ ]:
sess.set_lr(1e-2)

In [ ]:
sess.train(schedule, 3)

In [ ]:
sess.unfreeze()

In [ ]:
lr_find(sess, data['train'], start_lr=1e-5)

In [ ]:
sess.set_lr(5e-3)

In [ ]:
sess.train(schedule, 3)

In [ ]:
lr_scheduler.plot()